# Lab 3: Extending Logistic Regression
## by Michael Doherty, Leilani Guzman, and Carson Pittman

## 1. Business Understanding

In 1633, the Japanese government, feeling threatened by Spanish and Portugese influence within their country, banned foreign goods as part of their isolationist foreign policy (called [Sakoku](https://en.wikipedia.org/wiki/Sakoku)). As part of this ban, European playing cards were forbidden, so Japanese playing cards, called [hanafuda](https://lammuseum.wfu.edu/2021/12/japanese-hanafuda-cards/), were developed; however, hanafuda struggled to gain popularity among the Japanese populace. Centuries later, on November 22, 1859 in Kyoto, Japan, a man named Fusajiro Yamauchi was born. An aspiring artist and entrepreneur, Yamauchi saw an opportunity with hanafuda, so in 1889 he opened a shop to sell his own handcrafted cards. What was the name of that shop? [Nintendo Koppai](https://www.lifewire.com/fusajiro-yamauchi-founder-of-nintendo-729584) (which was later shortened to just Nintendo).

While Fusajiro Yamauchi would never know it, his card company would eventually become one of the biggest video game companies in the world. Since the mid-twentieth century, advancements in technology have propelled video games to become a multi-billion dollar industry. Yet, not all video games are made equal; from sports games like the [FIFA series](https://en.wikipedia.org/wiki/FIFA_(video_game_series)) to puzzle games like [Tetris](https://en.wikipedia.org/wiki/Tetris), there are several genres that a video game can belong to.

The dataset we've selected, titled "Global Video Game Sales", was created to analyze how a video game's genre relates to the platform it was released on, its publisher, and its sales (both globally and in various regions). The dataset consists of 16,600 instances with 11 multivariate attributes, comprised of numeric and categorical data types.

Ultimately, the prediction task for this dataset is to determine what a video game's genre is. Who might be interested in this research? For starters, video game companies may be interested to see which video game genre has the best sales. They may also find that certain genres are more popular in some markets than others (such as puzzle games being popular in Japan and sports games being popular in Europe). This information can help inform their strategies on what types of games they should make and where they should focus their marketing efforts. The prediction model created would be mostly used for offline analysis (as there aren't enough video games being created all the time that would justify having a deployed model).

So what does a good prediction algorithm for this dataset look like? There are several factors that need to be considered:
- **Finding Useful Trends**: One of the most important factors for the prediction algorithm is being able to find clear trends relating to a video game's genre. For example, finding that Nintendo games sell best in Japan, or that the most profitable video game genre is Shooter games, would be useful information that video game companies could use to make decisions regarding future video games, where to market their products more, etc.
- **Accuracy**: How accurate the prediction algorithm is at classifying video game genres is one of the most important aspects. The more accurate the prediction algorithm is, the more reliable the trends will be that we derive from the prediction algorithm. Thus, we want the accuracy to be as high as possible; at minimum, it should be higher than 50%, and ideally it would have  an accuracy greater than 90%.  

Thus, there are several aspects that need to be considered when trying to create a reliable prediction model. The data derived from the model should help companies make informed decisions on what types of video games to make, where they should advertise certain types of games more heavily, Ultimately, any good prediction model would need to prove useful to video game companies by providing useful insights that can help them increase their profits.

Link to the dataset: https://www.kaggle.com/datasets/thedevastator/global-video-game-sales

## 2. Data Preparation

In [ ]:
import pandas as pd

df = pd.read_csv("data/vgsales.csv")

df.head()

: 

In [ ]:
df.info()
rows_with_null_year = df[df["Year"].isna()]
rows_with_null_year

A lot of the missing data can be found in the dataset... (Madden NFL 2004 has multiple entries for each platform it was released on, and the other entries say it came out in 2003

In [ ]:
#Data Cleaning with Aggregation

df = df.groupby("Name").aggregate({'Rank': 'first', 'Platform': 'first', 'Name': 'first', 'Year': 'first', 'Genre': 'first', 'Publisher': 'last', 'NA_Sales': 'sum', 'EU_Sales': 'sum', 'JP_Sales': 'sum', 'Other_Sales': 'sum', 'Global_Sales': 'sum' })
del df["Platform"]
del df["Rank"]

df.info()

In [ ]:
#Data Cleaning Without Aggregation

df = df.dropna()
df.info()

In [ ]:
#One Hot Encoding

df_onehot = pd.get_dummies(df, columns=['Publisher', 'Genre'])

df_onehot.head()

: 

## 3. Modeling

In [ ]:
import numpy as np
from scipy.special import expit

# Binary Logistic Regression using vectorized implementation

class BinaryLogisticRegression:
    # Adjust default values for iterations and C as needed
    def __init__(self, eta, iterations=20, C=0.001, penalty='l2'):
        self.eta = eta          # The learning rate
        self.iterations = iterations
        self.C = C
        self.penalty = penalty
    def __str__(self):
        if(hasattr(self, 'weights_')):
            return "Binary Logistic Regression Object with coefficients:\n" + str(self.weights_)
        return 'Binary Logistic Regression Object is untrained'
    
    @staticmethod
    def _add_bias(X):
        # Add a column of float ones to X
        return np.hstack((np.ones((X.shape[0],1)), X))
    
    @staticmethod
    def _sigmoid(theta):
        return expit(theta)
    
    def _get_gradient(self, X, y):
        y_diff = y - self.predict_proba(X, add_bias=False).ravel()
        gradient = np.mean(X * y_diff[:,np.newaxis], axis = 0)
        gradient = gradient.reshape(self.weights_.shape)
        # Based on penalty type, add the appropriate term to the gradient
        if self.penalty == 'l1':
            # Double check this, not sure
            gradient[1:] += self.C * sum(abs(self.weights_[1:]))
        elif self.penalty == 'l2':
            # Double check this, not sure
            gradient[1:] += self.C * sum(self.weights_[1:]**2)
        elif self.penalty == 'both':
            gradient[1:] += self.C * sum(self.weights_[1:]**2) + self.C * sum(abs(self.weights_[1:]))
        elif self.penalty == 'none':
            pass
        return gradient
    
    def predict_proba(self, X, add_bias=True):
        print(type(X[0][0]))
        X_bias = self._add_bias(X) if add_bias else X
        return self._sigmoid(X_bias @ self.weights_)
    
    def predict(self, X):
        return (self.predict_proba(X) > 0.5)
    
    def fit(self, X, y):
        X_bias = self._add_bias(X)
        num_samples, num_features = X_bias.shape
        self.weights_ = np.zeros((num_features,1))    # Creating the weight vector
        print (self.weights_.shape)

        for _ in range(self.iterations):
            gradient = self._get_gradient(X_bias, y)
            self.weights_ += self.eta * gradient
    

In [ ]:
from numpy import ma
from scipy.optimize import minimize_scalar

## Optimizing 1: Steepest Ascent/Line Search

class LineSearchLogisticRegression(BinaryLogisticRegression):
    def __init__(self, line_iterations=0.0, **kwds):
        self.line_iterations = line_iterations
        super().__init__(**kwds)

        @staticmethod
        def objective_function(eta, X, y, w, gradient, C):
            w_new = w - gradient*eta
            gradient_new = expit(X @ w_new)
            return -np.sum(ma.log(g[y==1]))-ma.sum(ma.log(1-gradient_new[y==0])) + C*sum(w_new**2)
        
        def fit(self, X, y):
            X_bias = self._add_bias(X)
            num_samples, num_features = X_bias.shape
            self.weights_ = np.zeros((num_features,1))

            for _ in range (self.iterations):
                gradient = -self._get_gradient(X_bias, y)
                opts = {'maxiter': self.line_iterations}
                res = minimize_scalar(self.objective_function,
                                      bounds=(0,self.eta*10),
                                      args=(X_bias, y, self.weights_, gradient, self.C),
                                      method='bounded',
                                      options=opts)
                
            eta = res.x
            self.weights_ -= eta*gradient

In [ ]:
## Optimizing 2: Stoachastic Gradient Ascent

class StochasticLogisticRegression(BinaryLogisticRegression):
    def _get_gradient(self, X, y):
        # Adjust batch size as needed, calculates gradient according to this batch size
        batch_size = 16
        idxs = np.random.choice(len(y), batch_size)

        y_diff = y[idxs] - self.predict_proba(X[idxs], add_bias=False).ravel()
        gradient= np.mean(X[idxs] * y_diff[:,np.newaxis], axis = 0)

        gradient = gradient.reshape(self.weights_.shape)
        if self.penalty == 'l1':
            # Double check this, not sure
            gradient[1:] += self.C * sum(abs(self.weights_[1:]))
        elif self.penalty == 'l2':
            # Double check this, not sure
            gradient[1:] += self.C * sum(self.weights_[1:]**2)
        elif self.penalty == 'both':
            gradient[1:] += self.C * sum(self.weights_[1:]**2) + self.C * sum(abs(self.weights_[1:]))
        elif self.penalty == 'none':
            pass

        # This might need to be negative?
        return gradient

In [ ]:
from scipy.optimize import fmin_bfgs
## Optimizing 3: Quasi-Newton Method (BFGS)

class BFGSLogisticRegression(BinaryLogisticRegression):

    @staticmethod
    def objective_function(w, X, y, C):
        gradient = expit(X @ w)
        return -ma.sum(ma.log(g[y==1])-ma.sum(ma.log(1-g[y==0]))) + C*sum(w**2)
    
    @staticmethod
    def objective_gradient(self, w, X, y, C):
        gradient = expit(X @ w)
        y_diff = y-gradient
        gradient = np.mean(X * y_diff[:,np.newaxis], axis = 0)
        gradient = gradient.reshape(w.shape)
        if self.penalty == 'l1':
            # Double check this, not sure
            gradient[1:] += self.C * sum(abs(self.weights_[1:]))
        elif self.penalty == 'l2':
            # Double check this, not sure
            gradient[1:] += self.C * sum(self.weights_[1:]**2)
        elif self.penalty == 'both':
            gradient[1:] += self.C * sum(self.weights_[1:]**2) + self.C * sum(abs(self.weights_[1:]))
        elif self.penalty == 'none':
            pass
        return -gradient
    
    def fit(self, X, y):
        X_bias = self._add_bias(X)
        num_samples, num_features = X_bias.shape

        self.weights_ = fmin_bfgs(self.objective_function,
                                  np.zeros((num_features,1)),
                                  fprime=self.objective_gradient,
                                  args=(X_bias, y, self.C),
                                  gtol=1e-03,
                                  maxiter=self.iterations,
                                  disp=False)
        self.weights_ = self.weights_.reshape((num_features,1))

In [ ]:
#Regularized Logistic Regression using vectorized implementation
class MultiClassLogisticRegression:
    # Adjust default values for iterations and C as needed, temp default for solver is LineSearchLogisticRegression
    # Penalty can be either 'l1' or 'l2' or 'both', default to 'l2' for now (these are passed to the solver)
    def __init__(self, **kwds):
        self.eta = kwds.get('eta', 0.01)  # The learning rate
        self.iterations = kwds.get('iterations', 20)
        self.line_iterations = kwds.get('line_iterations', 0.0)
        self.C = kwds.get('C', 0.001)
        self.penalty = kwds.get('penalty', 'l2')
        self.solver = kwds.get('solver', LineSearchLogisticRegression)
        self.classifiers_ = []

    def __str__(self):
        if(hasattr(self, 'weights_')):
            return "Multiclass Logistic Regression Object with coefficients:\n" + str(self.weights_)
        return 'Multiclass Logistic Regression Object is untrained'
    
    def fit(self, X, y):
        num_samples, num_features = X.shape
        self.unique_classes_ = np.unique(y)
        num_unique_classes = len(self.unique_classes_)
        self.classifiers_ = []

        for i, y_value in enumerate(self.unique_classes_):
            # One vs All
            y_binary = (y==y_value)
            # Only pass line_iterations to solver if it is not 0.0
            if self.line_iterations == 0.0:
                lr = self.solver(eta=self.eta, iterations=self.iterations, C=self.C, penalty=self.penalty)
            else:
                lr = self.solver(eta=self.eta, iterations=self.iterations, line_iterations=self.line_iterations, C=self.C, penalty=self.penalty)
            lr.fit(X, y_binary)
            self.classifiers_.append(lr)
        
        self.weights_ = np.hstack([x.weights_ for x in self.classifiers_]).T

    def predict_proba(self, X):
        probabilities = []
        for lr in self.classifiers_:
            probabilities.append(lr.predict_proba(X))
        return np.hstack(probabilities)
    
    def predict(self, X):
        return self.unique_classes_[np.argmax(self.predict_proba(X), axis=1)]

In [ ]:
# Set X equal to the data in df, and y equal to the Genre column
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


X = df.drop(columns=['Genre'])
y = df['Genre']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Need to address the categorical variables in the data, represent them as binary variables somehow

# Extra parameter line_iterations only used for LineSearchLogisticRegression
# mlr = MultiClassLogisticRegression(eta=.5, 
#                                    iterations=4, 
#                                    line_iterations=0.0,
#                                    C=0.01, 
#                                    solver=LineSearchLogisticRegression, 
#                                    penalty='l2')

# mlr.fit(X_train, y_train)
# print(mlr)

# print('Accuracy: ', accuracy_score(y_test, mlr.predict(X_test)))

## 4. Deployment

## 5. Exceptional Work (rename to what we actually end up doing)